In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
import math
import random
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
with open('../data/text-1.txt', 'r', encoding='utf-8') as f:
    text = f.read().lower().replace('\n', ' ')

In [4]:
chars = sorted(set(text))

In [5]:
char2int = {c:(i+1) for i,c in enumerate(chars)}
# sot = '>'
# assert sot not in char2int
# char2int[sot] = 0
int2char = {i:c for c,i in char2int.items()}

In [9]:
ctxLen = 20
def getSamples():
    random.seed(123)
    samplNum = 100
    idxs = random.sample(range(0,len(text)-ctxLen), samplNum)
    X,Y = [],[]
    for i in idxs:
        X.append(text[i:i+ctxLen])
        Y.append(text[i+1:i+ctxLen+1])
    trSamplPct = 0.8
    i1 = int(trSamplPct*len(X))
    i2 = int((i1 + len(X)) / 2)
    Xtr,Ytr = X[:i1], Y[:i1]
    Xval,Yval = X[i1:i2], Y[i1:i2]
    Xtest,Ytest = X[i2:], Y[i2:]
    return (
        Xtr,Ytr,
        Xval,Yval,
        Xtest,Ytest,
    )
Xtr,Ytr, Xval,Yval, Xtest,Ytest = getSamples()

In [7]:
class Linear(nn.Module):
    def __init__(self,fanIn,fanOut,activation):
        super().__init__()
        self.lin = nn.Linear(fanIn,fanOut)
        self.act = activation
    def forward(self,x):
        y = self.lin(x)
        if self.act != None:
            y = self.act(y)
        return y

In [13]:
class MlpModel(nn.Module):
    def __init__(self,fanIn,sizes):
        super().__init__()
        self.layers = nn.Sequential(*[
            Linear(
                fanIn = fanIn if i == 0 else sizes[i-1], 
                fanOut = sizes[i], 
                activation = None if i == len(sizes)-1 else torch.nn.functional.leaky_relu
            ) for i in range(len(sizes))
        ])
    def forward(self,x):
        return self.layers(x)

In [20]:
model = MlpModel(fanIn=ctxLen, sizes=[100,ctxLen])
[p for p in model.parameters()]

[Parameter containing:
 tensor([[ 0.2016,  0.0750, -0.2085,  ...,  0.0093, -0.1055,  0.1632],
         [ 0.1994, -0.1762, -0.0265,  ..., -0.0241, -0.0112,  0.0294],
         [ 0.0014, -0.1645, -0.1499,  ...,  0.1917,  0.0826, -0.2090],
         ...,
         [ 0.1110,  0.2007, -0.0549,  ..., -0.0352, -0.1630,  0.2104],
         [-0.1522,  0.1235,  0.1966,  ...,  0.0996,  0.1539,  0.1738],
         [ 0.1937,  0.1628, -0.0520,  ...,  0.1617, -0.2083,  0.0608]],
        requires_grad=True),
 Parameter containing:
 tensor([-0.1752,  0.0749, -0.1147,  0.0146,  0.1331,  0.0701, -0.0721, -0.0218,
         -0.1663,  0.1065, -0.1549,  0.0618,  0.1168, -0.0312,  0.2109, -0.2118,
         -0.0812,  0.0775,  0.1026,  0.1929, -0.2138, -0.0749,  0.0709,  0.1014,
          0.0798, -0.1150, -0.2043, -0.0182,  0.0509, -0.1481, -0.0764,  0.1729,
         -0.1195,  0.0867,  0.1828,  0.0940, -0.1486, -0.1374,  0.0671, -0.1904,
          0.0755, -0.0583, -0.1308, -0.2054,  0.0737,  0.0764, -0.1411,  0.0545